In [7]:
import pyreadstat
import pandas as pd
enapres, meta = pyreadstat.read_sav("CAP_600_URBANO_7.sav")

In [8]:
import pandas as pd

# Filtrar las columnas relevantes
df_victimization = enapres[['NOMBREDI', 'P601_1', 'P601_2', 'P602C_2', 'P601_4A', 'P601_5A', 'P601_6A', 
                            'P601_7', 'P601_8', 'P601_9', 'P601_10', 'P601_12', 'P601_13', 
                            'P601_14', 'P601_15', 'P601_16']].copy()
df_victimization.rename(columns={'NOMBREDI': 'Distrito'}, inplace=True)

# Convertir todas las columnas de victimizaciones a valores numéricos
for col in df_victimization.columns[1:]:
    df_victimization[col] = pd.to_numeric(df_victimization[col], errors='coerce')

# Reemplazar respuestas: 1 = sí (víctima), 2 = no, 3 = no tiene
df_victimization.replace({1: 1, 2: 0, 3: None}, inplace=True)

# Eliminar filas con NaN en las columnas de victimizaciones
df_victimization.dropna(subset=df_victimization.columns[1:], inplace=True)

# Calcular el total de victimizaciones y el total de respuestas válidas
df_victimization['total_victimization'] = df_victimization.iloc[:, 1:].sum(axis=1)  
df_victimization['total_responses'] = df_victimization.iloc[:, 1:].count(axis=1)    

# Agrupar por distrito y calcular el porcentaje de victimización
df_rate = df_victimization.groupby('Distrito').apply(
    lambda x: (x['total_victimization'].sum() / x['total_responses'].sum() * 100) if x['total_responses'].sum() > 0 else None
).reset_index(name='porcentaje_victimizacion')

# Guardar el resultado en un archivo Excel
df_rate.to_excel('porcentaje_victimizacion_por_distrito.xlsx', index=False)

# Mostrar el resultado
print(df_rate)



                             Distrito  porcentaje_victimizacion
0                   ALTO SELVA ALEGRE                     12.50
1                            AMARILIS                     12.50
2   ANDRES AVELINO CACERES DORREGARAY                     18.75
3                           ASCENSION                     18.75
4                              ASCOPE                      6.25
..                                ...                       ...
87            VILLA MARIA DEL TRIUNFO                     12.50
88                               VIRU                     81.25
89                         YANACANCHA                     18.75
90                            YUNGUYO                    100.00
91                               YURA                     12.50

[92 rows x 2 columns]


C:\Users\Nicolas Silva\AppData\Local\Temp\ipykernel_20524\2646246854.py:24: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_rate = df_victimization.groupby('Distrito').apply(
